In [245]:
import sqlite3
import contextlib
import pandas as pd

In [246]:
data = pd.read_json("css-scraper-results.json")
data

,source,text,author,tags
0,http://quotes.toscrape.com/page/1/,“The world as we have created it is a process ...,Albert Einstein,"[change, deep-thoughts, thinking, world]"
1,http://quotes.toscrape.com/page/1/,"“It is our choices, Harry, that show what we t...",J.K. Rowling,"[abilities, choices]"
2,http://quotes.toscrape.com/page/1/,“There are only two ways to live your life. On...,Albert Einstein,"[inspirational, life, live, miracle, miracles]"
3,http://quotes.toscrape.com/page/1/,"“The person, be it gentleman or lady, who has ...",Jane Austen,"[aliteracy, books, classic, humor]"
4,http://quotes.toscrape.com/page/1/,"“Imperfection is beauty, madness is genius and...",Marilyn Monroe,"[be-yourself, inspirational]"
...,...,...,...,...
95,http://quotes.toscrape.com/page/10/,“You never really understand a person until yo...,Harper Lee,[better-life-empathy]
96,http://quotes.toscrape.com/page/10/,“You have to write the book that wants to be w...,Madeleine L'Engle,"[books, children, difficult, grown-ups, write,..."
97,http://quotes.toscrape.com/page/10/,“Never tell the truth to people who are not wo...,Mark Twain,[truth]
98,http://quotes.toscrape.com/page/10/,"“A person's a person, no matter how small.”",Dr. Seuss,[inspirational]


In [248]:
@contextlib.contextmanager
def dbconnection_mgr(file="Unit554.db"):
    con =  sqlite3.Connection(file)
    yield con
    con.close()

with dbconnection_mgr() as c:
    def saveRows(sql, dataTupleList):  # bulk inserts rows
        with c as cur:
            cur.executemany(sql, dataTupleList)
    
    def saveRow(sql, dataTuple):  # saves a single row and return the row id
        with c as cur:
            _cur = cur.cursor()
            _cur.execute(sql, dataTuple)
            return _cur.lastrowid
    
    def saveAuthors(data): # Saving unique Authors
        saveRows("insert into Authors('name') values(?) on conflict(name) do nothing;",  [(author,) for author in data["author"].unique()])
    
    def saveTags(data):  # Saving unique tags
        tags = pd.Series(dtype="str")
        for _tags in data["tags"]:
            tags = tags.append(pd.Series(_tags,dtype="str"))  
        saveRows("insert into Tags('name') values (?) on conflict(name) do nothing;",  [(tag,) for tag in tags.unique()])

    def saveAuthors_QuoteswithTags(data):
        quotes_Sql = "insert into Authors_Quotes(author_id, descr) values((select author_id from Authors where name=?), ?)";
        quotes_tags_Sql = "insert into Author_Quotes_Tagged (quote_id, tag_id) select ?, tag_id from tags where name in (?)";
        
        # Save Author quotes
        y = [x for x in data[["author", "text", "tags"]].items()] # returns an array of tupes. On tuple per column
        for author, quote, tags in zip(y[0][1], y[1][1], y[2][1]):
            quote_id = saveRow(quotes_Sql, (author, quote)) # Save Author quotes
            quote_tag_id = [(quote_id, tag) for tag in tags]
            saveRows(quotes_tags_Sql,quote_tag_id)  # Save Quotes and tags relationship
        
    """
      4 tables 
      Authors - store unique list of authors
      Tags - maintains unique list of tags
      Author_Quotes - maintains quotes by author id
      Author_Quotes_Tagged - maintains a relation of Author_quote_ids and tag ids
    """    
    
    saveAuthors(data)  # Save to Authors table - maintains unique list of Authors
    saveTags(data)     # Save to Tags table - maintains unique list of Tags
    saveAuthors_QuoteswithTags(data) # Save to Quotes for each author in table Authors_Quotes & maintains the quote and tag relationship in table Author_Quotes_Tagged
                                                                                       